In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`
┌ Info: Precompiling BpAlignGpu [5a3eb610-29b2-4cbe-8ba2-ea97f65fa95d]
└ @ Base loading.jl:1423


In [2]:
using CUDA
CUDA.device!(0)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/Uurn4/src/initialization.jl:70


CuDevice(0): TITAN RTX

In [3]:
#(q, N, L) = (21, 161, 67) 
(q, N, L) = (21, 5, 5) 

(21, 5, 5)

In [4]:
function convert_T3_xnTOy(tmp)
    SCE_ar = fill(NaN, 2N+4, L);
    for i=1:L
        for n=1:N+2
            SCE_ar[n, i] = tmp[n,1,i]
            SCE_ar[N+2+n, i] = tmp[n,2,i]
        end
    end
    return SCE_ar
end
function convert_T5_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L)
    for i=1:L
        for n1=1:N+2
            for n2=1:N+2
                SCE_ar[n1, n2, i] = tmp[n1, 1, n2, 1, i]
                SCE_ar[n1, N+2+n2, i] = tmp[n1, 1, n2, 2, i]
                SCE_ar[N+2+n1, n2, i] = tmp[n1, 2, n2, 1, i]
                SCE_ar[N+2+n1, N+2+n2, i] = tmp[n1, 2, n2, 2, i]
            end
        end
    end
    return SCE_ar
end
function convert_T6_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L, L)
    for i=1:L
        for j=1:L
            for n1=1:N+2
                for n2=1:N+2
                    SCE_ar[n1, n2, i, j] = tmp[n1, 1, n2, 1, i, j]
                    SCE_ar[n1, N+2+n2, i, j] = tmp[n1, 1, n2, 2, i, j]
                    SCE_ar[N+2+n1, n2, i, j] = tmp[n1, 2, n2, 1, i, j]
                    SCE_ar[N+2+n1, N+2+n2, i, j] = tmp[n1, 2, n2, 2, i, j]
                end
            end
        end
    end
    return SCE_ar
end

convert_T6_xnTOy (generic function with 1 method)

In [5]:
T = Float32

Float32

In [6]:
using Random
header = "myseq"
myseq = randstring('A':'Z', N)
ctype=Symbol("amino")
seq = Seq(header, myseq, ctype)

myseq
EHUFM

In [7]:
muint = 1.5
muext = 1.2
lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [8]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=5 N=5 q=21 size=43.547 KiB]

In [9]:
#build paramalgo
damp=T(0.0)
tol=T(1e-3)
tolnorm=T(1e-10)
tmax=10
upscheme=:random 
initcond=:random 
lr=:sce  
beta=T(1.0)
verbose=false
xnsol = fill((0, 0), L);
epscoupling=(false, T(0.0), xnsol)
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, initcond, lr, beta, verbose, epscoupling)

ParamAlgo{Float32}
-------------
damp=0.0
tol=0.001
tolnorm=1.0e-10
tmax=10
upscheme=random
initcond=random
lr=sce
beta=1.0
verbose=false
epscoupling=false
-------------

In [10]:
bpm = BPMessages(seq, pm, pa)

BPMessages{Float32}[L=5 N=5 ongpu=true size=20.820 KiB]

In [11]:
bpb = BPBeliefs(N, L)

BPBeliefs{Float32}[L=5 N=5 ongpu=true size=23.516 KiB]

In [12]:
lrf = LongRangeFields(N, L)

LongRangeFields{Float32}[L=5 N=5 ongpu=true size=4.102 KiB]

In [13]:
af = AllFields(bpm, bpb, lrf)

AllFields{Float32}[L=5 N=5 ongpu=true size=48.438 KiB]

# Checks

In [14]:
tmp_Jseq = Array(af.bpm.Jseq)
elts_Jseq = convert_T6_xnTOy(tmp_Jseq);

In [15]:
tmp_Hseq = Array(af.bpm.Hseq)
elts_Hseq = convert_T3_xnTOy(tmp_Hseq);

In [16]:
include("../src/SCE_update_messages.jl")
include("../src/SCE_update_shortrange_beliefs.jl")
include("../src/SCE_update_longrange.jl")

update_lr_free_nrj (generic function with 1 method)

# Check update f DCAlign

In [17]:
tmp_bel = Array(af.bpb.beliefs)
SCE_bel = convert_T3_xnTOy(tmp_bel);

In [18]:
BpAlignGpu.update_f_DCAlign!(af)

In [19]:
SCE_f = fill(NaN, 2N+4, L);

In [20]:
SCE_update_f_MF!(SCE_f, SCE_bel, J, N, L, seq.intseq)

In [21]:
i=4
af.lrf.f[:,:,i]

7×2 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.162053  0.0
 0.420546  0.26271
 0.703819  0.398848
 0.911149  0.703819
 1.34654   1.18466
 1.57865   1.62941
 1.62437   0.0

In [22]:
SCE_f[:,i]

14-element Vector{Float64}:
   0.1620531178629508
   0.420546032178944
   0.703818697128345
   0.9111487023011605
   1.3465364593076583
   1.5786451696539143
   1.6243729419563457
 -Inf
   0.26271025694860495
   0.3988480853636081
   0.703818697128345
   1.184657635903323
   1.6294063083456707
 -Inf

# check BP updates

In [23]:
#BpAlignGpu.update_hF!(af, pm, pa)
#BpAlignGpu.update_hB!(af, pm, pa)
#BpAlignGpu.update_F!(af, pm, pa)
BpAlignGpu.update_B!(af, pm, pa)

In [24]:
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

#tmp_g = Array(af.lrf.g)
#SCE_g = convert_T5_xnTOy(tmp_g);
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

SCE_B = zeros(2*(N+2),L);

In [25]:
i=2
#SCE_update_F!(SCE_F, i, damp, beta, tolnorm, SCE_hF, SCE_f, muext, muint, N, elts_Hseq)
SCE_update_B!(SCE_B, i, damp, beta, tolnorm, SCE_hB, SCE_f, muext, muint, N, L, elts_Hseq)
#SCE_update_hF!(SCE_hF, i, SCE_F, damp, beta, tolnorm, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)
#SCE_update_hB!(SCE_hB, i, damp, beta, tolnorm, SCE_B, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)

In [26]:
af.bpm.B[:,:,i]

7×2 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0269749  0.0
 0.0212458  0.0717165
 0.114175   0.276471
 0.0248986  0.248398
 0.0447041  0.135723
 0.0147712  0.00976404
 0.0111587  0.0

In [27]:
SCE_B[:,i]

14-element Vector{Float64}:
 0.026974874940110253
 0.02124576899252568
 0.11417466963562518
 0.024898590223555193
 0.04470414060337473
 0.014771200679868082
 0.01115866929629876
 0.0
 0.07171652990772416
 0.27647061362100644
 0.24839798491137935
 0.13572291756442487
 0.009764039624107407
 0.0

# check update beliefs

In [28]:
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

SCE_bel = zeros(2*(N+2),L);

In [29]:
BpAlignGpu.update_beliefs!(af, pm, pa)

In [30]:
i=5
SCE_update_beliefs!(SCE_bel, i, beta, tolnorm, SCE_hF, SCE_hB, SCE_f, muint, muext, N, L, elts_Hseq)

1.3123673649251923

In [31]:
af.bpb.beliefs[:,:,i]

7×2 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0       0.0
 0.0       0.0629179
 0.0       0.197784
 0.0       0.126108
 0.0       0.00444651
 0.0       0.0085766
 0.600167  0.0

In [32]:
SCE_bel[:,i]

14-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.6001672987780243
 0.0
 0.06291794728084141
 0.1977838690164353
 0.1261077711286273
 0.004446515307386164
 0.00857659848868548
 0.0

In [30]:
logzi, mynorm_Zi = BpAlignGpu.logZi(af, pm, pa)

(-7.6260653f0, [-0.65365124;;; -2.2202148;;; -2.2581332;;; -2.6339424;;; 0.13987625])

In [31]:
mynorm_Zi[i]

┌ Warning: Performing scalar indexing on task Task (runnable) @0x00007f196fec1b60.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArrays /home/louise/.julia/packages/GPUArrays/umZob/src/host/indexing.jl:56


0.13987625f0

# check update joint proba

In [32]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);

tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

SCE_joint = zeros(2(N+2), 2(N+2), L);

In [33]:
BpAlignGpu.update_jointchain!(af, pm, pa)

In [34]:
i=4
af.bpb.joint_chain[:,1,:,1,i]

7×7 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.00298098

In [35]:
SCE_update_joint_chain!(SCE_joint, i, beta, tolnorm, SCE_F, SCE_B, SCE_g, lambda_o, lambda_e, N, elts_Jseq)

0.06536807561481568

In [36]:
rx1 = 1:N+2
rx2 = N+3:2N+4
SCE_joint[rx1,rx1, i]

7×7 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.00298098

In [37]:
logza, mynormza = BpAlignGpu.logZa(af, pm, pa)

(-1.2062328f0, [-0.55355954;;;;; -0.033462103;;;;; -0.68457925;;;;; 0.065368034;;;;; 0.0])

In [38]:
mynormza[i]

0.065368034f0

# Check logZedge

In [39]:
BpAlignGpu.logZia(af, pm)

-19.023766f0

In [40]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

In [41]:
logZedge(SCE_F, SCE_B, SCE_hF, SCE_hB)

(-9.594253540205155, -9.429512034096916, -19.02376557430207)

# check update conditional on the chain

In [42]:
BpAlignGpu.update_conditional_chain!(af, pa)

In [43]:
tmp_joint = Array(af.bpb.joint_chain)
SCE_joint = convert_T5_xnTOy(tmp_joint);

In [44]:
cond_chain = fill(0.0, 2N+4, 2N+4, L, L)
for i=1:L-1
    SCE_update_conditional_chain!(i, cond_chain, SCE_joint, tolnorm, N)
end

In [45]:
af.bpb.conditional[:,2,:,1,1,2]

7×7 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0       0.0       0.0       0.0       0.0       0.0
 0.0  0.370141  0.0       0.0       0.0       0.0       0.0962225
 0.0  0.0       0.619319  0.0       0.0       0.0       0.135307
 0.0  0.0       0.0       0.673274  0.0       0.0       0.463564
 0.0  0.0       0.0       0.0       0.471372  0.0       0.133553
 0.0  0.0       0.0       0.0       0.0       0.266593  0.0861734
 0.0  0.0       0.0       0.0       0.0       0.0       0.0

In [46]:
cond_chain[rx2, rx1, 1, 2]

7×7 Matrix{Float64}:
 0.0  0.0       0.0       0.0       0.0       0.0       0.0
 0.0  0.370141  0.0       0.0       0.0       0.0       0.0962225
 0.0  0.0       0.619319  0.0       0.0       0.0       0.135307
 0.0  0.0       0.0       0.673274  0.0       0.0       0.463564
 0.0  0.0       0.0       0.0       0.471372  0.0       0.133553
 0.0  0.0       0.0       0.0       0.0       0.266593  0.0861734
 0.0  0.0       0.0       0.0       0.0       0.0       0.0

# check update conditional off the chain

In [47]:
BpAlignGpu.update_conditional_all!(af, pm);

In [48]:
cond_lr = copy(cond_chain);
pointer(cond_chain), pointer(cond_lr)

(Ptr{Float64} @0x0000000011495140, Ptr{Float64} @0x0000000035195a80)

In [49]:
SCE_update_conditional_lr!(cond_lr)

In [50]:
cond_lr[rx1, rx2, 4, 1]

7×7 Matrix{Float64}:
 0.0  0.0          0.0         0.0        0.0        0.0       0.0
 0.0  0.000413058  0.0         0.0        0.0        0.0       0.0
 0.0  0.00846363   8.66591e-5  0.0        0.0        0.0       0.0
 0.0  0.040853     0.0102258   0.0018733  0.0        0.0       0.0
 0.0  0.0963792    0.148821    0.0580933  0.0247416  0.0       0.0
 0.0  0.0610765    0.10298     0.222946   0.291714   0.231122  0.0
 0.0  0.248231     0.300433    0.558071   0.620127   0.768878  0.0

In [51]:
af.bpb.conditional[:,1,:,2,4,1]

7×7 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0          0.0         0.0        0.0        0.0       0.0
 0.0  0.000413058  0.0         0.0        0.0        0.0       0.0
 0.0  0.00846363   8.66591f-5  0.0        0.0        0.0       0.0
 0.0  0.040853     0.0102258   0.0018733  0.0        0.0       0.0
 0.0  0.0963792    0.148822    0.0580932  0.0247416  0.0       0.0
 0.0  0.0610764    0.10298     0.222946   0.291714   0.231122  0.0
 0.0  0.248231     0.300433    0.55807    0.620127   0.768878  0.0

# Check long range Free Energy

In [52]:
BpAlignGpu.lr_freeen(af, pm)

4.451576769351959

In [53]:
tmp_bel = Array(af.bpb.beliefs)
SCE_bel = convert_T3_xnTOy(tmp_bel)
tmp_cond = Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [54]:
using LinearAlgebra
update_lr_free_nrj(SCE_bel, SCE_cond, elts_Jseq)

4.451576755178928

# check update f

In [ ]:
BpAlignGpu.update_F!(af, pm, pa)
BpAlignGpu.update_hF!(af, pm, pa)
BpAlignGpu.update_B!(af, pm, pa)
BpAlignGpu.update_hB!(af, pm, pa)
BpAlignGpu.update_beliefs!(af, pm, pa)
BpAlignGpu.update_jointchain!(af, pm, pa)
BpAlignGpu.update_conditional_chain!(af, pa)
BpAlignGpu.update_conditional_all!(af, pm)

In [ ]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [ ]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

In [ ]:
BpAlignGpu.update_f!(af)

In [ ]:
using LinearAlgebra
SCE_update_f!(SCE_f, SCE_cond, elts_Jseq, N)

In [ ]:
i=3
af.lrf.f[:,:,i]

In [ ]:
SCE_f[:,i]

# check update g

In [ ]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [ ]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

In [ ]:
BpAlignGpu.update_g!(af)

In [ ]:
SCE_update_g!(SCE_g, SCE_cond, elts_Jseq, N)

In [ ]:
i=3
af.lrf.g[:,1,:,2,i]

In [ ]:
rx1 = 1:N+2
rx2 = N+3:2N+4
SCE_g[rx1,rx2,i]